# Table of Contents
<p>
<div class="lev1 toc-item">
    <a href="#Cargamos-la-serie" data-toc-modified-id="Cargamos-la-serie">
        <span class="toc-item-num">1&nbsp;&nbsp;</span>
        Cargamos la serie</a></div>

<div class="lev1 toc-item">
    <a href="#Gráfica-de-ubicaciones" data-toc-modified-id="Gráfica-de-ubicaciones">
    <span class="toc-item-num">2&nbsp;&nbsp;</span>
    Gráfica de ubicaciones</a></div>

In [1]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import folium
from folium.plugins import MarkerCluster, HeatMapWithTime
%matplotlib inline

## Cargamos la serie

In [32]:
df = pd.read_csv(os.path.join('./data/Serie_Total2016.csv'))

In [33]:
df.timestamp = pd.to_datetime(df.timestamp)

In [34]:
# Eliminamos los valores de ocupación 0 correspondientes a las 20 horas
df = df.loc[df.timestamp.dt.hour < 20]

In [35]:
# Fijo un rango de una hora y un día del año
import datetime
fixed_hour = 12
fixed_day = datetime.datetime(2016,7,15).timetuple().tm_yday
subset_df = df.loc[(df.timestamp.dt.hour == fixed_hour) & (df.timestamp.dt.dayofyear == fixed_day)]

In [52]:
# Seleccionamos los 100 primeros element_key
df_first100 = subset_df.loc[:,['latitude','longitude','occupation_perc']].head(100)

## Mapa de porcentaje de ocupación por parquímetro

In [57]:
this_map = folium.Map(prefer_canvas=True, max_bounds=False)

def plotDot(df):
    df = df.copy()
#     df['color'] = coloring_function(df.quadrant) #.map(coloring_function)
    folium.CircleMarker(location=[df.latitude, df.longitude],
                        radius= 10,
#                        weight=df.occupation_perc,
                        popup=str(df.occupation_perc),
                        fill=True,
                        fill_opacity=0.8,
                        fill_color=usedColor).add_to(this_map)

usedColor='red'
df_first100.loc[df_first100.occupation_perc >= 75].apply(plotDot, axis = 1)
usedColor='orange'
df_first100.loc[(df_first100.occupation_perc >= 50) & (df_first100.occupation_perc < 75)].apply(plotDot, axis = 1)
usedColor='yellow'
df_first100.loc[(df_first100.occupation_perc >=25) & (df_first100.occupation_perc < 50)].apply(plotDot, axis = 1)
usedColor='green'
df_first100.loc[(df_first100.occupation_perc < 25)].apply(plotDot, axis = 1)

map_bounds = this_map.get_bounds()
this_map.fit_bounds(map_bounds, max_zoom=20)
this_map.max_lat = map_bounds[1][0]
this_map.min_lat = map_bounds[0][0]
this_map.max_lon = map_bounds[1][1]
this_map.min_lon = map_bounds[0][1]

this_map.save('seatle_parkimeters_occup_perc_example.html')